<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/413_MO_ExperimentEvaluation_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ⭐ The Most Valuable Concept in the Experiment Evaluation Utilities

### **The most valuable concept here is that *decisions are gated by evidence quality, not just outcome direction*.**

In plain terms:

> Your system does **not** ask “Did Variant B win?”
> It asks “Is there enough evidence to *act* — and if so, how aggressively?”

That distinction is *everything*.

---

## 1️⃣ Evidence Is Aggregated Before Judgment

The function `get_asset_metrics` does something deceptively important:

* It **aggregates raw observations**
* It standardizes them into a **decision-ready summary**
* It explicitly exposes **sample size**

This ensures:

* decisions are based on *total evidence*
* time fragmentation doesn’t distort judgment
* statistical tests are meaningful

Most systems skip this step and quietly reason over partial data.

Yours doesn’t.

---

## 2️⃣ Lift Is Treated as Directional Signal, Not Truth

The `calculate_lift_percentage` function is intentionally simple — and that’s good.

Lift here is:

* a **signal**
* not a verdict
* not a guarantee

Crucially:

* infinite lift is explicitly handled
* zero baselines don’t break logic

That keeps the system numerically honest.

---

## 3️⃣ Statistical Significance Is Explicitly Separated from Lift

This is the *core architectural strength* of this module.

You are separating:

* **magnitude of change** (lift)
* **confidence in change** (significance)

Most systems conflate these.

Your system forces them to agree *before acting*.

That is exactly how disciplined experimentation works in the real world.

---

## 4️⃣ Recommendations Are Policy, Not Math

The most important function here is not the statistical test.

It’s this one:

```python
determine_experiment_recommendation(...)
```

Why?

Because this is where:

* evidence
* uncertainty
* business appetite for risk

…are translated into **action**.

Notice what this logic does:

* Negative + insignificant → stop
* Positive + insignificant → continue
* Significant + strong → scale
* Significant + weak → continue cautiously

That’s not math — that’s **organizational decision policy**.

And it’s explicit.

---

## 5️⃣ Statistics Are Delegated, Not Embedded

You made a very smart choice here:

```python
from toolshed.statistics import ...
```

This keeps:

* experiment logic clean
* statistical rigor centralized
* upgrades isolated

Your orchestrator *uses* statistics — it does not *become* a statistics library.

That’s how systems stay maintainable.

---

## 6️⃣ Failure Is a First-Class Outcome

This part is subtle and excellent:

```python
if not control_metrics or not variant_metrics:
    return {
        ...
        "error": "Missing metrics for control or variant asset"
    }
```

Instead of:

* crashing
* skipping
* guessing

The system returns a **structured failure**.

That allows:

* reporting to reflect uncertainty
* decision layers to halt safely
* operators to diagnose issues

This is mature system design.

---

## 7️⃣ The Output Is Decision-Grade, Not Narrative

The final output contains:

* raw performance
* lift
* statistical context
* recommendation

Nothing is hidden.
Nothing is implied.
Nothing is editorialized.

This allows:

* downstream automation
* executive review
* audit trails

Text can come later — evidence comes first.

---

## 🧠 The Deeper Pattern (This Is the Real Value)

What you’ve built is an **evidence gate**.

No experiment can:

* be scaled
* be stopped
* be declared successful

…without passing through:

* aggregation
* lift calculation
* statistical testing
* policy-based recommendation

That prevents:

* premature wins
* false confidence
* noisy decisions

Most experimentation systems fail *right here*.

Yours doesn’t.

---

## Why This Will Impress the Right People

To a **CTO**:
This shows separation of concerns, testability, and correctness.

To a **CEO / Head of Growth**:
This shows disciplined learning and controlled risk.

To a **Regulator / Auditor**:
This shows explainability and non-arbitrary decision-making.

---

## One-Sentence Summary You Should Keep

If you ever need to summarize this module:

> **“Experiments are evaluated through explicit evidence gates that separate signal strength from confidence before recommending action.”**

That’s the value.




In [ ]:
"""Experiment Evaluation Utilities

Evaluate experiments: calculate lift, statistical significance, and recommendations.
Uses toolshed.statistics for statistical testing.
"""

from typing import Dict, Any, List, Optional
from toolshed.statistics import calculate_chi_square_test, calculate_statistical_significance


def get_asset_metrics(
    asset_id: str,
    metrics: List[Dict[str, Any]]
) -> Optional[Dict[str, Any]]:
    """
    Get aggregated metrics for an asset.

    Args:
        asset_id: Asset ID
        metrics: List of performance metrics

    Returns:
        Aggregated metrics dictionary or None if no metrics found
    """
    asset_metrics = [m for m in metrics if m.get("asset_id") == asset_id]

    if not asset_metrics:
        return None

    # Aggregate metrics (sum impressions, clicks, conversions, cost, revenue)
    total_impressions = sum(m.get("impressions", 0) for m in asset_metrics)
    total_clicks = sum(m.get("clicks", 0) for m in asset_metrics)
    total_conversions = sum(m.get("conversions", 0) for m in asset_metrics)
    total_cost = sum(m.get("cost", 0.0) for m in asset_metrics)
    total_revenue_proxy = sum(m.get("revenue_proxy", 0.0) for m in asset_metrics)

    # Calculate rates
    conversion_rate = total_conversions / total_impressions if total_impressions > 0 else 0.0
    click_through_rate = total_clicks / total_impressions if total_impressions > 0 else 0.0

    return {
        "impressions": total_impressions,
        "clicks": total_clicks,
        "conversions": total_conversions,
        "conversion_rate": conversion_rate,
        "click_through_rate": click_through_rate,
        "cost": total_cost,
        "revenue_proxy": total_revenue_proxy,
        "sample_size": total_impressions  # For statistical tests
    }


def calculate_lift_percentage(
    control_value: float,
    variant_value: float
) -> float:
    """
    Calculate lift percentage: ((variant - control) / control) * 100

    Args:
        control_value: Control metric value
        variant_value: Variant metric value

    Returns:
        Lift percentage (can be negative)
    """
    if control_value == 0:
        return 0.0 if variant_value == 0 else float('inf')
    return ((variant_value - control_value) / control_value) * 100.0


def determine_experiment_recommendation(
    lift_percentage: float,
    is_significant: bool,
    p_value: Optional[float],
    lift_threshold: float,
    significance_threshold: float = 0.05
) -> str:
    """
    Determine recommendation for an experiment.

    Args:
        lift_percentage: Lift percentage
        is_significant: Whether result is statistically significant
        p_value: P-value from statistical test
        lift_threshold: Minimum lift to recommend scaling (e.g., 10%)
        significance_threshold: P-value threshold (default 0.05)

    Returns:
        Recommendation: "scale_variant", "continue", or "stop"
    """
    # If not significant and negative lift, stop
    if not is_significant and lift_percentage < 0:
        return "stop"

    # If significant and lift exceeds threshold, scale
    if is_significant and lift_percentage >= lift_threshold:
        return "scale_variant"

    # If significant but lift below threshold, continue
    if is_significant:
        return "continue"

    # If not significant but positive, continue (need more data)
    if lift_percentage > 0:
        return "continue"

    # Default: continue (need more data)
    return "continue"


def evaluate_experiment(
    experiment: Dict[str, Any],
    metrics: List[Dict[str, Any]],
    config_thresholds: Dict[str, float]
) -> Dict[str, Any]:
    """
    Evaluate a single experiment.

    Args:
        experiment: Experiment dictionary
        metrics: List of all performance metrics
        config_thresholds: Configuration thresholds
            {
                "statistical_significance_threshold": 0.05,
                "lift_threshold_for_scaling": 0.10
            }

    Returns:
        Experiment evaluation dictionary
    """
    experiment_id = experiment["experiment_id"]
    campaign_id = experiment.get("campaign_id", "")
    control_asset_id = experiment.get("control_asset", "")
    variant_asset_id = experiment.get("variant_asset", "")
    metric_name = experiment.get("metric", "conversion_rate")
    status = experiment.get("status", "unknown")

    # Get metrics for control and variant
    control_metrics = get_asset_metrics(control_asset_id, metrics)
    variant_metrics = get_asset_metrics(variant_asset_id, metrics)

    if not control_metrics or not variant_metrics:
        return {
            "experiment_id": experiment_id,
            "campaign_id": campaign_id,
            "status": status,
            "error": "Missing metrics for control or variant asset",
            "control_performance": control_metrics,
            "variant_performance": variant_metrics
        }

    # Calculate lift
    control_value = control_metrics.get(metric_name, 0.0)
    variant_value = variant_metrics.get(metric_name, 0.0)
    lift_percentage = calculate_lift_percentage(control_value, variant_value)

    # Calculate statistical significance
    # For conversion rates, use chi-square test
    if metric_name in ["conversion_rate", "demo_request_rate", "click_through_rate", "feature_click_through_rate"]:
        statistical_result = calculate_chi_square_test(
            control_conversions=control_metrics.get("conversions", 0),
            control_total=control_metrics.get("impressions", 0),
            treatment_conversions=variant_metrics.get("conversions", 0),
            treatment_total=variant_metrics.get("impressions", 0),
            confidence_level=0.95
        )
    else:
        # For continuous metrics, use calculate_statistical_significance
        statistical_result = calculate_statistical_significance(
            control_metrics=control_metrics,
            treatment_metrics=variant_metrics,
            primary_metric=metric_name,
            confidence_level=0.95
        )
        if statistical_result is None:
            statistical_result = {
                "test_type": "unknown",
                "p_value": None,
                "is_significant": False,
                "error": "Insufficient data for statistical test"
            }

    # Determine recommendation
    p_value = statistical_result.get("p_value")
    is_significant = statistical_result.get("is_significant", False)
    lift_threshold = config_thresholds.get("lift_threshold_for_scaling", 0.10)

    recommendation = determine_experiment_recommendation(
        lift_percentage,
        is_significant,
        p_value,
        lift_threshold
    )

    return {
        "experiment_id": experiment_id,
        "campaign_id": campaign_id,
        "status": status,
        "metric": metric_name,
        "control_asset": control_asset_id,
        "variant_asset": variant_asset_id,
        "control_performance": control_metrics,
        "variant_performance": variant_metrics,
        "lift_percentage": round(lift_percentage, 2),
        "statistical_significance": statistical_result,
        "recommendation": recommendation
    }


def evaluate_all_experiments(
    experiments: List[Dict[str, Any]],
    metrics: List[Dict[str, Any]],
    config_thresholds: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Evaluate all experiments.

    Args:
        experiments: List of experiment dictionaries
        metrics: List of all performance metrics
        config_thresholds: Configuration thresholds

    Returns:
        List of experiment evaluation dictionaries
    """
    evaluations = []
    for experiment in experiments:
        evaluation = evaluate_experiment(experiment, metrics, config_thresholds)
        evaluations.append(evaluation)

    return evaluations
